## 1. Import Libraries

In [352]:
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import *
from pyspark.sql.functions import col, count, sum


## 2. Load Data and Convert to Spark Data Frame

In [353]:
# All column names
sc.textFile("loan.csv").take(1)

[u'id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m']

In [354]:
# load data as dataframe
loan_df = spark.read.csv("loan.csv",header=True)
loan_df.rdd.getNumPartitions()
type(loan_df)

pyspark.sql.dataframe.DataFrame

Select useful columns to create smaller dataframe for future use

## 3. Create response variable and features

### 3.1 Remove some columns based on EDA results

In [355]:
loan_df1 = loan_df.drop('desc','mths_since_last_delinq','mths_since_last_record','next_pymnt_d',
                        'mths_since_last_major_derog','annual_inc_joint','dti_joint','verification_status_joint',
                        'open_acc_6m','open_il_6m','open_il_12m','open_il_24m','mths_since_rcnt_il','total_bal_il',
                        'il_util','open_rv_12m','open_rv_24m','max_bal_bc','all_util','inq_fi','total_cu_tl',
                        'inq_last_12m', # with a lot NA
                        'id','member_id','collection_recovery_fee','last_pymnt_amnt','last_pymnt_d','out_prncp','out_prncp_inv',
                        'pymnt_plan','recoveries','term','title','total_pymnt','total_pymnt_inv','total_rec_int',
                        'total_rec_late_fee','total_rec_prncp','url','verification_status', 'initial_list_status', 
                        'last_credit_pull_d','policy_code','emp_title','last_credit_pull_d' # domain knowledge
                       )

In [308]:
loan_df1.head()

Row(loan_amnt=u'5000.0', funded_amnt=u'5000.0', funded_amnt_inv=u'4975.0', int_rate=u'10.65', installment=u'162.87', grade=u'B', sub_grade=u'B2', emp_length=u'10+ years', home_ownership=u'RENT', annual_inc=u'24000.0', issue_d=u'Dec-2011', loan_status=u'Fully Paid', purpose=u'credit_card', zip_code=u'860xx', addr_state=u'AZ', dti=u'27.65', delinq_2yrs=u'0.0', earliest_cr_line=u'Jan-1985', inq_last_6mths=u'1.0', open_acc=u'3.0', pub_rec=u'0.0', revol_bal=u'13648.0', revol_util=u'83.7', total_acc=u'9.0', collections_12_mths_ex_med=u'0.0', application_type=u'INDIVIDUAL', acc_now_delinq=u'0.0', tot_coll_amt=None, tot_cur_bal=None, total_rev_hi_lim=None)

In [309]:
len(loan_df1.columns)

30

In [326]:
# Convert left columns into right format
loan_df2 = loan_df.select(
    loan_df.loan_amnt.cast("integer"),
    loan_df.funded_amnt.cast("integer"),
    loan_df.funded_amnt_inv.cast("integer"),
    loan_df.int_rate.cast("integer"),
    loan_df.installment.cast("integer"),
    'grade',
    #'sub_grade',
    #'emp_title',
    'emp_length',
    'home_ownership',
    loan_df.annual_inc.cast("integer"),
    # 'issue_d',
    'loan_status', # response variable
    'purpose',
    # 'zip_code',
    # 'addr_state',
    loan_df.dti.cast("integer"),
    loan_df.delinq_2yrs.cast("integer"),  # Above done-Maxine
    # 'earliest_cr_line',
    loan_df.inq_last_6mths.cast("integer"),
    loan_df.open_acc.cast("integer"),
    loan_df.pub_rec.cast("integer"),
    loan_df.revol_bal.cast("integer"),
    loan_df.revol_util.cast("integer"),
    loan_df.total_acc.cast("integer"),
    #loan_df.last_credit_pull_d.cast("integer"),
    'application_type',
    loan_df.acc_now_delinq.cast("integer"),
    loan_df.tot_coll_amt.cast("integer"),
    loan_df.tot_cur_bal.cast("integer"),
    loan_df.total_rev_hi_lim.cast("integer")
)


### 3.2 Create response variable and remove rows with no valid response variable

In [327]:
def whetherpaid(x):
    if x in ['Default', 'Charged Off', 'Does not meet the credit policy. Status:Charged Off']:
        return 0
    elif x in ['Does not meet the credit policy. Status:Fully Paid', 'Fully Paid']:
        return 1
    else:
        return -1

In [328]:
paidflag = udf(lambda x: whetherpaid(x))

In [329]:
loan_df3 = loan_df2.withColumn('paid_flag',paidflag('loan_status')).where("paid_flag != -1").drop('loan_status')

### 3.3 Create features

#### 3.3.1 Creating a category feature for "Loan Purpose"

There are 14 types of loan purpose. StringIndexer encodes a string column of labels to a column of label indices, and most frequent label gets index 0.

In [330]:
indexer = StringIndexer(inputCol="purpose", outputCol="purposeIndex")
loan_df4 = indexer.fit(loan_df3).transform(loan_df3).drop('purpose')

####  3.3.2 Create a numeric feature for "emp_length"

In [331]:
import re
def convert_to_int(s):
    s = re.sub('\\D', '', s)
    
    try:
        return s
    except ValueError:
        return 'NaN'

emp_to_num = udf(convert_to_int)
loan_df5 = loan_df4.withColumn('emp_len',emp_to_num('emp_length')).drop('emp_length')

In [332]:
loan_df5.take(1)

[Row(loan_amnt=5000, funded_amnt=5000, funded_amnt_inv=4975, int_rate=11, installment=163, grade=u'B', home_ownership=u'RENT', annual_inc=24000, dti=28, delinq_2yrs=0, inq_last_6mths=1, open_acc=3, pub_rec=0, revol_bal=13648, revol_util=84, total_acc=9, application_type=u'INDIVIDUAL', acc_now_delinq=0, tot_coll_amt=None, tot_cur_bal=None, total_rev_hi_lim=None, paid_flag=u'1', purposeIndex=1.0, emp_len=u'10')]

#### 3.3.3 Create numeric variable for grade

In [333]:
# count how many nulls in each column
def count_null(c):
    return sum(col(c).isNull().cast("integer")).alias(c)

exprs = [count_null(c) for c in loan_df5.columns[0:9]]
loan_df5.agg(*exprs).show()

+---------+-----------+---------------+--------+-----------+-----+--------------+----------+---+
|loan_amnt|funded_amnt|funded_amnt_inv|int_rate|installment|grade|home_ownership|annual_inc|dti|
+---------+-----------+---------------+--------+-----------+-----+--------------+----------+---+
|        0|          0|              0|       0|          0|    0|             0|         4|252|
+---------+-----------+---------------+--------+-----------+-----+--------------+----------+---+



In [334]:
loan_df5.groupBy('grade').count().show()

+-----+-----+
|grade|count|
+-----+-----+
|    F| 7986|
|    E|19923|
|    B|76617|
|    D|41509|
|    C|66309|
|    A|42441|
|    G| 2154|
+-----+-----+



In [335]:
indexer = StringIndexer(inputCol="grade", outputCol="gradeIndex")
loan_df6 = indexer.fit(loan_df5).transform(loan_df5).drop('grade')

#### 3.3.4 Create numeric variable for home_ownership

In [336]:
loan_df6.groupBy('home_ownership').count().show()

+--------------+------+
|home_ownership| count|
+--------------+------+
|           OWN| 22282|
|          RENT|107831|
|      MORTGAGE|126598|
|           ANY|     1|
|         OTHER|   179|
|          NONE|    48|
+--------------+------+



In [337]:
def home_ownership_func(x):
    if x in ['ANY','OTHER','NONE']:
        return 'Other'
    else: 
        return x

home_ownership = udf(home_ownership_func)
loan_df7 = loan_df6.withColumn('homeownership',emp_to_num('home_ownership')).drop('home_ownership')

In [338]:
indexer = StringIndexer(inputCol="homeownership", outputCol="homeownershipIndex")
loan_df8 = indexer.fit(loan_df7).transform(loan_df7).drop('homeownership')

In [339]:
len(loan_df8.columns)
loan_df8.take(1)

[Row(loan_amnt=5000, funded_amnt=5000, funded_amnt_inv=4975, int_rate=11, installment=163, annual_inc=24000, dti=28, delinq_2yrs=0, inq_last_6mths=1, open_acc=3, pub_rec=0, revol_bal=13648, revol_util=84, total_acc=9, application_type=u'INDIVIDUAL', acc_now_delinq=0, tot_coll_amt=None, tot_cur_bal=None, total_rev_hi_lim=None, paid_flag=u'1', purposeIndex=1.0, emp_len=u'10', gradeIndex=0.0, homeownershipIndex=0.0)]

#### Replace missing values of annual income and dti with mean value

In [340]:
loan_df9 = loan_df8.fillna(0.0, ['annual_inc','dti']).where('delinq_2yrs is not null')

In [361]:
exprs = [count_null(c) for c in loan_df9.columns[0:8]]
loan_df9.agg(*exprs).show()

+---------+-----------+---------------+--------+-----------+----------+---+-----------+
|loan_amnt|funded_amnt|funded_amnt_inv|int_rate|installment|annual_inc|dti|delinq_2yrs|
+---------+-----------+---------------+--------+-----------+----------+---+-----------+
|        0|          0|              0|       0|          0|         0|  0|          0|
+---------+-----------+---------------+--------+-----------+----------+---+-----------+



#### Add loan_inc_ratio feature

In [344]:
def calculate_ratio(a, b):
    try:
        return a/float(b)
    except TypeError:
        return 'NaN'
    except ZeroDivisionError:
        return 'NaN'

ratio = udf(calculate_ratio)

In [346]:
loan_df10 = loan_df9.withColumn('loan_inc_ratio',ratio('loan_amnt','annual_inc').cast('float'))

#### Add feature instal_inc_ratio

In [356]:
def calculate_monthly_ratio(a, b):
    try:
        return a/(float(b)/12)
    except TypeError:
        return 'NaN'
    except ZeroDivisionError:
        return 'NaN'
    
monthly_ratio = udf(calculate_monthly_ratio)

In [357]:
loan_df11 = loan_df10.withColumn('instal_inc_ratio',ratio('installment','annual_inc').cast('float'))

In [358]:
loan_df11.printSchema()

root
 |-- loan_amnt: integer (nullable = true)
 |-- funded_amnt: integer (nullable = true)
 |-- funded_amnt_inv: integer (nullable = true)
 |-- int_rate: integer (nullable = true)
 |-- installment: integer (nullable = true)
 |-- annual_inc: integer (nullable = true)
 |-- dti: integer (nullable = true)
 |-- delinq_2yrs: integer (nullable = true)
 |-- inq_last_6mths: integer (nullable = true)
 |-- open_acc: integer (nullable = true)
 |-- pub_rec: integer (nullable = true)
 |-- revol_bal: integer (nullable = true)
 |-- revol_util: integer (nullable = true)
 |-- total_acc: integer (nullable = true)
 |-- application_type: string (nullable = true)
 |-- acc_now_delinq: integer (nullable = true)
 |-- tot_coll_amt: integer (nullable = true)
 |-- tot_cur_bal: integer (nullable = true)
 |-- total_rev_hi_lim: integer (nullable = true)
 |-- paid_flag: string (nullable = true)
 |-- purposeIndex: double (nullable = true)
 |-- emp_len: string (nullable = true)
 |-- gradeIndex: double (nullable = true)

### 4. Modeling

Currently this part was written using python, will re-write using MLlib later

In [362]:
loan_df_final = loan_df11.select('loan_amnt','funded_amnt','funded_amnt_inv',
                                 'int_rate','annual_inc','dti','delinq_2yrs',
                                'loan_inc_ratio','instal_inc_ratio','paid_flag')